# Parse development book

### Notebook purpose
This notebook is development space for python parse.ts replacement and upgrade.
It reads specified google sheets and output actants.json file, which can be imported to inkVisitor RethinkDB.py

The aggreated doc for dev: https://docs.google.com/document/d/1ga6R_9TWAQlXE9XqPE_qZ2S8S8aVW2A7n8SuYpSXnoI/edit#heading=h.q9ntf0ofam2u
The holy schema: https://app.diagrams.net/#G1bKvqEKr6JzPWryVg-vYudQy_4KvuzHS9

### JSon schemas for the actants
 * are created from *.ts files through typescript-jschema commandline utility
 * the schemas are trasformed as python Classes through warlock library


### Principles of the parsing operations
 * There are google sheet dataset, which need to be transformed to json format according to inkVisior datamodel
 * inkVisitor holds datamodel in the code, the typescript classes (see /shared/types...), or the holy schema
 * DZ created in input table parsing instructions
   * the first 5 rows contain instructions
   * if the keywords contain ? character, they are ignored (it is work in progress from DZ)

#### Parsing instructions
 * explicit set of keywords
   * discard
     * ...
   * inside
     * the value in the column should be straitforwadly made part of the entity object, the exists update_generic method or update_fieldName mehthods for fields with custom but generally applicable logic
   * propvalue
     * for making so called
          * statement or property statment (old A0093 has relation)
     * it sits in the entity "props" attribute, it has IProp class
   * special
     * fully custom method for the parsing behavior
     *

### Parsing process
 1. Prepare data model entities classes (from typescript inkvisitor classes -> json schema ->  python classes)
 2. Load and wrangle all input data
 3. Process headers of the tables for parsing instructions
 4. Process tables row by row, column by column
 5. Save as json file (which can be imported to the inkVisitor RethinkDB)


# DEV Update January 2023
encapsulation of code


# DEV Updates : November
 - TODO updated model for relations (attribute order)
 - TODO synonymic relations parsed to "cloud"

# Dev updates:  October 2
- DONE change event-territories generation,  genre is relation.classification, not metaprop
- TODO prepare for "partial parsing" and fixed guids for "done" tables
- DONE updated users
- DONE change in Manuscript parsing, For each non-empty, non-NA value: check whether the script hasn't already generated an L entity with the same label.
IF NOT: (1) Generate a new L entity with label = value in this col., status = "approved", entity logical type = "definite", label language = value in the next col. (repository_label_language); (2) append to this L entity a RelationType.Classification leading to C2646 "archive or library". (3) Go to No. 4.
IF YES, go to No. 4 and use that L.
(4) Under the O entity representing the row (i.e. the physical manuscript), add a metaprop which will relate this O to this L entity (repository) through the relation: O - (has) - C2645 "repository" - L in this col.




# Dev updates:  October
 - DONE u action a conceptu se neparsuji relations ?
 - DONE revize skriptu vzhledem k september bordelu
 - DONE events parsing
 - DONE 21.09. 2022, R table has received a new parsable column “components_id”. Its notation follows our agreed-upon notation, so it should not pose any problem, but I rather signal this new col. to you. It is for composite Rs, which need to be separated into several identities with different URIs (typically two volumes of an edition)
  - DONE The instruction here and here for generating creation events for Territories and manuscript Objects has changed, since what was the Property C0565 class is now the RelationType Classification. Please take this change into account, tx.
 - DONE territories generation
  - light yellow: metadata of Subterritory capturing the lowest-level document recording the Event (not of Event described by the row). Tomáš H.: (1) Please generate these lowest-level T entities following Adam’s script for parsing Robert’s Sellan coding sheet IDs into a hierarchical T structure and put them under their proper root T through legacy IDs: first element of ID = root T ID, last element = lowest-level Territory whose label follows in the next col., middle elements = folders in between root and lowest child level. E.g. here, in Guglielmites, there is root, i.e. the whole Guglielmites process of Beltramo Salvagno; then parts 1-4; and underneath them, the individual documents. When creating these lowest-level Subterritories, use this attributes: Entity type = T; T label and label_language defined in the two respective cols. which follow; status = approved. Other metadata set as metaprops in the following yellow cols. (2) Generate the middle entities holding the hierarchy, name them by the chunk ID (in this case, this will be four Ts named “1”, “2”, “3”, and “4”), and relate the lowest-level entity to them as their parents, and relate those middle entities to the root T as their parent in turn; attributes of these middle entities (folders 1-4 of the records): label = number [1,2,3,4] depending on the chunk; legacy id = T107-[chunk number], label language = Latin, status = approved. (3) Relate the lowest-level entity to the Event described by the row by the following Metaproperty: T107-1-01 - [has] - C2286 represented event - E0001 (middle-level entities will have no such relation now, only lowest-level documents).


# Dev updates:  September
 - adding new users

# Dev updates:  August
 - new instruction with proptype_2nd in column
    - before we got the type fixed and had just instruction propvalue_2nd (used in territories for manuscript witness)
 - new instruction "hook-inside" for insides in newly created entities (see in gugli locations special:modern_name)
 - changes :  some properties 'id' changed to 'specificId'
   - IProp {... 'value':{'entityId' and 'type':{'entityId'
   - IStatement {  'data':{ 'territory' {'territoryId'
   - ITerritory { 'data': {'parent': {'territoryId': ""
- bigger TODO relations

# Dev updates: April

## Concepts
  * DONE wordnet_synset_id
    * wordnet_id bounded to resource object R0067

## Texts
 * DONE special_creation_event_id
   * instructions:
     * Create entities in this col. as new E entities, with (1) the value here as legacy_id, (2) assign (as usual) a new "hash" ID from the db, (3) label of this E: see next col., (4) logical type "definite" (default), (5) label language "English", (6) status "approved", and (7) attach to any of those Es the metaprop "(has) - C0565 "class" - C2642 "creation" (to instantiate the event to its event type = event class).
   * creation_event_label in standalone field
     * this will trigger the operation
   * lot of proptype and propvalue with non-standard schema: new branch of interpreting

## Manuscripts
 * special_repository_label
   * i:
     * For each non-empty, non-NA, non-NS row: (1) generate L entity with label = value in this col., status = "approved", entity logical type = "definite", label language = value in the next col. (repository_label_language); (2) append to this L entity a metaprop (has) - C0565 "class" - C2646 "archive or library", and (3) under the O entity representing the row (i.e. the physical manuscript), add a metaprop which will relate this O to this L entity (repository) through the relation: O - (has) - C2645 "repository" - L in this col.
   * repository_label_language
     * i:
       * Use this value as label language value of the repository L entity.
 * DONE creation_event_id
   * creation_event_label
   * i:
     * Create entities in this col. as new E entities, with (1) the value here as legacy_id, (2) assign (as usual) a new "hash" ID from the db, (3) label of this E: see next col., (4) logical type "definite" (default), (5) label language "English", (6) status "approved", and (7) attach to any of those Es the metaprop "(has) - C0565 "class" - C2642 "creation" (to instantiate the event to its event type = event class).
 * reproduction_online_url
   * instructions:
     * If non-empty, non-NA, (1) generate an R entity with label "Reproduction of " + label of the MS (i.e. value in the B column, status = "approved", label-language = "English", url = the URL sitting under the hyperlink value in this cell, and (2) add metaprop to the O entity represented by this row: O - (has) - C1199 "digital reproduction" - the R entity here generated.

### Input variables

In [2]:
input_tables = ["texts", "manuscripts", "resources", "actions" , "concepts"]

#                   sheet_name,  gdrive code, header_in_row, columns which need suffix based on a case name
input_sheets = {
    "texts" : ("Texts","13eVorFf7J9R8YzO7TmJRVLzIIwRJS737r7eFbH1boyE", 5), #https://docs.google.com/spreadsheets/d/13eVorFf7J9R8YzO7TmJRVLzIIwRJS737r7eFbH1boyE/edit#gid=2056508047
    "manuscripts" : ("Manuscripts", "13eVorFf7J9R8YzO7TmJRVLzIIwRJS737r7eFbH1boyE", 5),
    "resources" : ("Resources", "13eVorFf7J9R8YzO7TmJRVLzIIwRJS737r7eFbH1boyE", 5),
    "actions" :  ("Statements","1vzY6opQeR9hZVW6fmuZu2sgy_izF8vqGGhBQDxqT_eQ", 5), # https://docs.google.com/spreadsheets/d/1vzY6opQeR9hZVW6fmuZu2sgy_izF8vqGGhBQDxqT_eQ/edit#gid=0
    "concepts" : ("Concepts","1nSqnN6cjtdWK-y6iKZlJv4iGdhgtqkRPus8StVgExP4", 5), # https://docs.google.com/spreadsheets/d/1nSqnN6cjtdWK-y6iKZlJv4iGdhgtqkRPus8StVgExP4/edit#gid=0,
    "R0006_persons" : ("Persons","1UtgKvjr91egdgOQMP8WxsRxgGDT9FC92qBG3sKBiYvk", 5, ['id','residence_id','social_relation1_value_id','social_relation2_value_id','social_relation3_value_id']), # GUGLIELMITES Persons, https://docs.google.com/spreadsheets/d/1UtgKvjr91egdgOQMP8WxsRxgGDT9FC92qBG3sKBiYvk/edit#gid=0,
    "R0007_locations" : ("Locations","1jImuYCTu-QrZc7oCT1Vz01SLu5sxYRo4xzJzlF2J8Bs", 5, ['id','locationrelation1_value_id','superordinate_location_id']) ,# GULIELMITES Locations, https://docs.google.com/spreadsheets/d/1jImuYCTu-QrZc7oCT1Vz01SLu5sxYRo4xzJzlF2J8Bs/edit#gid=0,
    "R0008_events" : ("Events","1mLevawnoZVLQv6qINaSfiyi1NNlfJzTZevinRSSQVPo", 5, ['id','spatial_ref_value_id','participant_id','inquisitor_id','notary_id','witness_assessor_id','accuser_id','accused_id','mentioned_id','denied_id','marked_as_incriminated_id','temporal_ref_1_value_id','temporal_ref_2_value_id']),
    # Guglielmites EVENTS https://docs.google.com/spreadsheets/d/1mLevawnoZVLQv6qINaSfiyi1NNlfJzTZevinRSSQVPo/edit#gid=0,
    "R0075_persons" : ("Persons","1Tyi4DYnHDeTs8tuz9rxN3wyq1LqLWWp_ZPgOimQoxf8", 5, ['id','origin_or_residence_id','residence_id','relations_1_value_id','relations_2_value_id','relations_3_value_id','relations_4_value_id','relations_5_value_id','relations_6_value_id','relations_7_value_id','relations_8_value_id','relations_9_value_id','relations_10_value_id','relations_11_value_id','relations_12_value_id','relations_13_value_id','origin_id','occupation_id','office_id']), # Piedmont Castellario Persons, https://docs.google.com/spreadsheets/u/1/d/1Tyi4DYnHDeTs8tuz9rxN3wyq1LqLWWp_ZPgOimQoxf8/edit#gid=0
    "R0035_locations" : ("Locations","1VwUDnAMJ8mWvL-54K-OnZmguB-rALnJjtGG2IGNBqIU", 5, ['id','locationrelation1_value_id','locationrelation2_value_id','identificate_id','superordinate_location_id']) ,# Piedmont Castellario Locations, https://docs.google.com/spreadsheets/d/1VwUDnAMJ8mWvL-54K-OnZmguB-rALnJjtGG2IGNBqIU/edit#gid=0
    "R0083_events" : ("Events","1M5uhJsbjUCxWP_avYQM0fr1Hm0GXVmYuDvqSoEuUAbs", 5, ['id','spatial_ref_value_id','participant_id','inquisitor_id','notary_id','witness_assessor_id','accuser_id','accused_id','mentioned_id','denied_id','temporal_ref_1_value_id','temporal_ref_2_value_id']),
    # Castellario Incriminations https://docs.google.com/spreadsheets/u/1/d/1M5uhJsbjUCxWP_avYQM0fr1Hm0GXVmYuDvqSoEuUAbs/edit#gid=0
     "R0105_groups" : ("Groups","1IqislE4C0GM45Wvlg9JW2QmzqvR7LgjA5TZTkM143U4", 5, ['id','has_member_id']) ,# Piedmont Castellario Groups, https://docs.google.com/spreadsheets/d/1IqislE4C0GM45Wvlg9JW2QmzqvR7LgjA5TZTkM143U4/edit#gid=0


    "R0030_persons" : ("Persons","1W8YTiKBZZOi_682guwH7N9aEX_qINi9D3qBM8u96Fhk", 5, ['id','residence_id', 'origin_id','social_relation1_value_id','social_relation2_value_id','social_relation3_value_id','social_relation4_value_id','social_relation5_value_id']), # Bologna Persons, https://docs.google.com/spreadsheets/d/1W8YTiKBZZOi_682guwH7N9aEX_qINi9D3qBM8u96Fhk/edit#gid=0
    "R0087_groups" : ("Groups","1WuTIsyBaC7aD_o8xtS3VcZ8_lmA5T-SABKPIkHyWsPw", 5, ['id']), # Bologna Groups, https://docs.google.com/spreadsheets/d/1WuTIsyBaC7aD_o8xtS3VcZ8_lmA5T-SABKPIkHyWsPw/edit#gid=0
    "R0031_locations" : ("Locations","1AvGxtm3fSg9sl8CrjKiLaLLwTFaerSiz3wBUUjl2Il8", 5, ['id','superordinate_location_id','province_modern_id','locationrelation1_value_id']) ,# Bologna Locations, https://docs.google.com/spreadsheets/d/1AvGxtm3fSg9sl8CrjKiLaLLwTFaerSiz3wBUUjl2Il8
    "R0082_events" : ("Events","1qYTMaY-DwI2XiFTLUL9tiGC7ah9Lkm0YeKWXw6V0yog", 5, ['id','participant_id','inquisitor_id','notary_id','witness_assessor_id','spatial_ref_value_id','accuser_id','accused_id','mentioned_id','denied_id']), # Bologna Trial Events, https://docs.google.com/spreadsheets/d/1qYTMaY-DwI2XiFTLUL9tiGC7ah9Lkm0YeKWXw6V0yog/edit#gid=0

    "R0039_persons" : ("Persons","1jV9RBSZHSsKiMpJIrB58YdsaPxjBe3RxNLpR1ODJyWU", 5, ['id','residence_id','origin_id','origin_or_residence_id','relation1_value_id','relation2_value_id','relation3_value_id','relation4_value_id', 'property_value_id']), # Carcassone HHH Persons, https://docs.google.com/spreadsheets/d/1jV9RBSZHSsKiMpJIrB58YdsaPxjBe3RxNLpR1ODJyWU/edit#gid=0

    # NO GROUPS in Carcassonne

    "R0040_locations" : ("Locations","1FSLbf0IE4zpchuHL1hHOsJEx0jgYGSoHU5RZzt86jLI", 5, ['id','superordinate_location_id','locationrelation1_target_id','locationrelation2_target_id']) ,# Carcassone HHH  Locations, https://docs.google.com/spreadsheets/d/1FSLbf0IE4zpchuHL1hHOsJEx0jgYGSoHU5RZzt86jLI/edit#gid=0
    "R0079_events" : ("Events","1i0XTsxvhP0vkHXoWEDTNtpuQIg-lWNY7jSOyjMZMHbw", 5, ['id','person_before_court_id','inquisitor_id','notary_id','witness_assessor_id','spatial_ref_value_id','accuser_id','accused_id','mentioned_id','denied_id']), # Carcassone HHH  Trial Events, https://docs.google.com/spreadsheets/d/1i0XTsxvhP0vkHXoWEDTNtpuQIg-lWNY7jSOyjMZMHbw/edit#gid=0

    "R0076_persons" : ("Persons","1yIT3kK9OdslT5FykzTjV1kDPkNVZfhIuayzqTK3hgA4", 5, ['id','relations_1_value_id','relations_2_value_id','origin_or_residence_id','residence_id','origin_id']), # Casasco Persons, https://docs.google.com/spreadsheets/d/1yIT3kK9OdslT5FykzTjV1kDPkNVZfhIuayzqTK3hgA4/edit#gid=0
    "R0074_locations" : ("Locations","1lUKyrZu9r9do9zXjzF7GUhMDgihXVYszwuOA5imuCDc", 5, ['id','locationrelation1_value_id','locationrelation2_value_id','superordinate_location_id','identificate_id']) ,# Casasco  Locations, https://docs.google.com/spreadsheets/d/1lUKyrZu9r9do9zXjzF7GUhMDgihXVYszwuOA5imuCDc/edit#gid=0
    "R0104_events" : ("Events","1XQCBAG__LXnYXmsCiDq-l6iO9cXZTyr4yXV0ISAm-sQ", 5, ['id','participant_id','inquisitor_id','notary_id','witness_id','spatial_ref_value_id','temporal_ref_1_value_id','temporal_ref_2_value_id','accuser_id','accused_id','mentioned_id','denied_id']) # Casasco Trial Events, https://docs.google.com/spreadsheets/d/1XQCBAG__LXnYXmsCiDq-l6iO9cXZTyr4yXV0ISAm-sQ/edit#gid=0
}


# table of tables
# https://docs.google.com/spreadsheets/d/13eVorFf7J9R8YzO7TmJRVLzIIwRJS737r7eFbH1boyE/edit#gid=583750775


# resource IDs
# R0006 Guglielmites Persons
# R0007	Guglielmites Locations
# R0008	Guglielmites Events

# R0035	Piedmont Locations Castellario
# R0075	Persons Castellario
# R0083 Castellario Incriminations
# R0105 Castellario Groups

# RXXXX     Castellario Groups, https://docs.google.com/spreadsheets/d/1IqislE4C0GM45Wvlg9JW2QmzqvR7LgjA5TZTkM143U4/edit#gid=0

# R0087	Bologna Groups
# R0031	Bologna Locations
# R0030	Bologna Persons
# R0082	Bologna Trial Events
# https://docs.google.com/spreadsheets/d/1WuTIsyBaC7aD_o8xtS3VcZ8_lmA5T-SABKPIkHyWsPw/edit#gid=0
# https://docs.google.com/spreadsheets/d/1AvGxtm3fSg9sl8CrjKiLaLLwTFaerSiz3wBUUjl2Il8
# https://docs.google.com/spreadsheets/d/1W8YTiKBZZOi_682guwH7N9aEX_qINi9D3qBM8u96Fhk/edit#gid=0
# https://docs.google.com/spreadsheets/d/1qYTMaY-DwI2XiFTLUL9tiGC7ah9Lkm0YeKWXw6V0yog/edit#gid=0

# R0040	Carcassonne HHH Locations
# R0039	Carcassonne HHH Persons
# R0079	Carcassonne HHH Trial Events
# https://docs.google.com/spreadsheets/d/1FSLbf0IE4zpchuHL1hHOsJEx0jgYGSoHU5RZzt86jLI/edit#gid=0
# https://docs.google.com/spreadsheets/d/1jV9RBSZHSsKiMpJIrB58YdsaPxjBe3RxNLpR1ODJyWU/edit#gid=0
# https://docs.google.com/spreadsheets/d/1i0XTsxvhP0vkHXoWEDTNtpuQIg-lWNY7jSOyjMZMHbw/edit#gid=0

# R0074	Casasco Locations
# R0076	Casasco Persons
# R0104	Casasco Trial Events
# https://docs.google.com/spreadsheets/d/1lUKyrZu9r9do9zXjzF7GUhMDgihXVYszwuOA5imuCDc/edit#gid=0
# https://docs.google.com/spreadsheets/d/1yIT3kK9OdslT5FykzTjV1kDPkNVZfhIuayzqTK3hgA4/edit#gid=0
# https://docs.google.com/spreadsheets/d/1XQCBAG__LXnYXmsCiDq-l6iO9cXZTyr4yXV0ISAm-sQ/edit#gid=0


table_to_entity = {
    "concepts" : "IConcept",
    "resources" : "IResource",
    "texts" : "ITerritory",
    "manuscripts" : "IObject",
    "actions" :  "IAction",
    "R0006_persons" : "IPerson",
    "R0007_locations": "ILocation",
    "R0008_events": "IEvent",
    "R0075_persons" : "IPerson",
    "R0035_locations": "ILocation",
    "R0083_events": "IEvent",
    "R0105_groups": "IGroup",
    "R0030_persons" : "IPerson",
    "R0087_groups" : "IGroup",
    "R0031_locations": "ILocation",
    "R0082_events": "IEvent",
    "R0039_persons" : "IPerson",
    "R0040_locations": "ILocation",
    "R0079_events": "IEvent",
    "R0076_persons" : "IPerson",
    "R0074_locations": "ILocation",
    "R0104_events": "IEvent",
}

root_sheet_url = "https://docs.google.com/spreadsheets/d/"
google_api_dotenv_path = "../env/.env.googleapi"  # contains google api specs for sheet access with Dator

### Libraries

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import os, warlock, json
from datetime import datetime
import time
from jsonschema import validate
import dissinetpytools.dator as dator
from dotenv import load_dotenv

import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)
#warnings.filterwarnings("ignore")

import numpy as np
from copy import deepcopy
from shutil import copyfile

import uuid
import dill as pickle
from DataHolder import DataHolder
import inkimplib


# type hinting
from collections.abc import Sequence, Callable
from typing import List, Dict, Tuple


### Initialisation

In [5]:
load_dotenv(google_api_dotenv_path) # fills os.environ['GDRIVE_API_CREDENTIALS']
d = dator.Dator(loglevel=10, print_log_online=True, cache=True, project_name="inkvisitor-import") # expects 'GDRIVE_API_CREDENTIALS' in the global system variables (os.environ)
d.google_authenticate()
logger = d.logger

20 2023-01-19 09:27:24 : Google authentification start
20 2023-01-19 09:27:24 : Google authentification end
20 2023-01-19 09:27:24 : Dator initiation succesfull end


In [285]:
# factory for making entity objects, contains defaults with "prerequisities"

IOF = inkimplib.InkVisitorJSONObjectFactory("../schemas/","")
IOF.validate_defaults()

2023-01-21 17:53:45,681 INFO Class IActant available.
2023-01-21 17:53:45,682 INFO Class IAction available.
2023-01-21 17:53:45,683 INFO Class IAudit available.
2023-01-21 17:53:45,684 INFO Class IConcept available.
2023-01-21 17:53:45,685 INFO Class IEntity available.
2023-01-21 17:53:45,686 INFO Class IEvent available.
2023-01-21 17:53:45,687 INFO Class IGroup available.
2023-01-21 17:53:45,688 INFO Class ILabel available.
2023-01-21 17:53:45,688 INFO Class ILocation available.
2023-01-21 17:53:45,689 INFO Class IObject available.
2023-01-21 17:53:45,690 INFO Class IPerson available.
2023-01-21 17:53:45,691 INFO Class IProp available.
2023-01-21 17:53:45,692 INFO Class IReference available.
2023-01-21 17:53:45,692 INFO Class IResource available.
2023-01-21 17:53:45,694 INFO Class IStatement available.
2023-01-21 17:53:45,694 INFO Class ITerritory available.
2023-01-21 17:53:45,695 INFO Class IUser available.
2023-01-21 17:53:45,696 INFO Class IValue available.
2023-01-21 17:53:45,697

In [7]:
# initialisation of input data tables
tables = {}
header_infos = {}
entity_ids = {}

# Load input datasets

In [284]:
# partial load
# partial_load = ['R0075_persons', 'R0035_locations','R0083_events']
# partial_load = ['R0035_locations']
# partial_load = ['R0039_persons', 'R0040_locations','R0079_events']
# partial_load = ['R0076_persons', 'R0074_locations','R0104_events']

# R0087	Bologna Groups
# R0031	Bologna Locations
# R0030	Bologna Persons
# R0082	Bologna Trial Events
# partial_load = ['R0030_persons', 'R0031_locations','R0082_events']
# partial_load = ['R0104_events'] # 'R0030_persons', 'R0087_groups', ,'R0082_events'
partial_load = [] # 'R0030_persons', 'R0087_groups', ,'R0082_events'

# empty value unifier
def unify_empty_value(df: pd.DataFrame, empty_values=None, unified_empty_value =''):
    if empty_values is None:
        empty_values = ["#N/A", "#VALUE!",'NA', 'NS'] # 'NA', 'NS',
    for naner in empty_values:
        df = df.replace(naner,unified_empty_value, regex=True)
    df. fillna(unified_empty_value, inplace=True)
    return df

# define cross-suffixes
casearea_to_entity_suffix = {
    "R0006" : {
        "P" : "R0006",
        "L" : "R0007",
        "E" : "R0008"
    },
    "R0007" : {
        "P" : "R0006",
        "L" : "R0007",
        "E" : "R0008"
    },
    "R0008" : {
        "P" : "R0006",
        "L" : "R0007",
        "E" : "R0008"
    },
    "R0075" : {
        "P" : "R0075",
        "L" : "R0035",
        "E" : "R0083",
        "G" : "R0105",
    },
    "R0035" : {
        "P" : "R0075",
        "L" : "R0035",
        "E" : "R0083",
        "G" : "R0105",
    },
    "R0083" : {
        "P" : "R0075",
        "L" : "R0035",
        "E" : "R0083",
        "G" : "R0105",
    },
    "R0105" : {
        "P" : "R0075",
        "L" : "R0035",
        "E" : "R0083",
        "G" : "R0105",
    },
    "R0087" : {
          "P" : "R0030",
          "L" : "R0031",
          "E" : "R0082",
          "G" : "R0087",
      },
    "R0030" : {
        "P" : "R0030",
        "L" : "R0031",
        "E" : "R0082",
        "G" : "R0087",
    },
    "R0031" : {
        "P" : "R0030",
        "L" : "R0031",
        "E" : "R0082",
        "G" : "R0087",
    },
    "R0082" : {
        "P" : "R0030",
        "L" : "R0031",
        "E" : "R0082",
        "G" : "R0087",
    },
    "R0039" : {
          "P" : "R0039",
          "L" : "R0040",
          "E" : "R0079",
      },
    "R0040" : {
          "P" : "R0039",
          "L" : "R0040",
          "E" : "R0079",
      },
    "R0079" : {
          "P" : "R0039",
          "L" : "R0040",
          "E" : "R0079",

      },
    "R0076" : {
          "P" : "R0076",
          "L" : "R0074",
          "E" : "R0104"
      },
    "R0074" : {
          "P" : "R0076",
          "L" : "R0074",
          "E" : "R0104"
      },
     "R0104" : {
          "P" : "R0076",
          "L" : "R0074",
          "E" : "R0104"
      },
}

def get_suffix_based_on_case_and_entity_type(string, case_suffix):
    # if valid entity type
    allowed_strict_entities = ['P','L','E','B','G']
    if any(string.startswith(c)for c in allowed_strict_entities):
        fl = string[0]
        return "_"+casearea_to_entity_suffix[case_suffix.replace("_","")][fl]
    else:
        return ""

def apply_suffix(string, case_suffix):
    if string in ['NS','NA']:
        return string
    mod_string = ""
    if "#" in string:
        strings = string.split("#")
        for s in strings:
            mod_string += " #"+ s.strip() + get_suffix_based_on_case_and_entity_type(s.strip(), case_suffix)
        mod_string = mod_string[2:]
    else:
        mod_string = string + get_suffix_based_on_case_and_entity_type(string, case_suffix)
    return mod_string


# load all input tables
load_sheets = {}
if len(partial_load) > 0:
    logger.info(f"Making PARTIAL LOAD with {partial_load}")
    for item in partial_load:
        load_sheets[item] = input_sheets[item]
else:
    load_sheets = input_sheets

for key, sheet in load_sheets.items():
    logger.info(f"Calling for {key} with sheet_name {sheet[0]}.")
    tables[key], header_infos[key] = d.load_df_from_gsheet(key,root_sheet_url + sheet[1], sheet[0], fromCache=False, header_in_row=sheet[2], clean=True, fillna=True, cleanByColumn="label", parse_hyperlink_formulas=True, numerize=False)
    tables[key] = unify_empty_value(tables[key])
    header_infos[key] = unify_empty_value(header_infos[key])

    # clean brackets
    for c in tables[key].columns:
      if "label" not in c:
        tables[key][c] = tables[key][c].replace("\[","",  regex=True)
        tables[key][c] = tables[key][c].replace("\]","",  regex=True)


    # case study dataset legacyId needs a suffix
    if "_" in key:
        columns = sheet[3]
        suffix = "_" + key.split("_")[0]
        for c in columns:
            # tables[key][c] = tables[key][c] + suffix
            logger.info(f"Using {key} and {c}.")
            tables[key][c]  = tables[key][c].apply(lambda item: apply_suffix(item,suffix) if len(str(item)) > 0 else item)

            #tables[key].loc[~tables[key][c].isna(), "c"] = tables[key][c] + "_" + suffix
            # df.loc[df["gender"] == "male", "gender"] = 1
            logger.info(f"{key} For columns {c} the suffix from {suffix} set was glued.")

    # code for legacyId copy and uuid creation
    # tables[key]['legacyId'] = tables[key]['id'].copy()
    tables[key].insert(2, 'legacyId', tables[key]['id'].copy())

    # inform instructive header about the new column and what to do with it
    header_infos[key]['legacyId'] = ""
    header_infos[key].at[3,'legacyId'] = "inside"
    tables[key]['id'] = tables[key].apply(lambda x: inkimplib.get_uuid_id(), axis=1)  # generate unique id for each row
    # make id dictionaries (it is much faster to search for keys there in legacyId>id retrievals)
    ed = tables[key][["legacyId","id"]].set_index("legacyId")

    if "_" in key:   # legacyIds are case-specified, i.e. all locations can be together in one table
        key = key.split("_")[1]

    if key in  entity_ids:
        entity_ids[key] = {**entity_ids[key],**ed["id"].to_dict()}
    else:
        entity_ids[key] = ed["id"].to_dict()

2023-01-21 17:47:33,546 INFO Calling for texts with sheet_name Texts.


20 2023-01-21 17:47:34 : Loading dataset texts
20 2023-01-21 17:47:34 : Opting for variant header at row 5.
20 2023-01-21 17:47:37 : Hyperlinks were detected and transformed in columns ['edition_1', 'edition_2', 'edition_3', 'persons_index_link', 'places_index_link'].
20 2023-01-21 17:47:37 : Dropping empty columns in the dataset texts : (1011, 115)
20 2023-01-21 17:47:38 : Deleted 858 empty rows by label.
20 2023-01-21 17:47:38 : Loaded and prepared dataset texts : (153, 115)
20 2023-01-21 17:47:38 : Making pickle cache of  texts with separeted header file : (153, 115)


2023-01-21 17:47:38,185 INFO Calling for manuscripts with sheet_name Manuscripts.


20 2023-01-21 17:47:38 : Loading dataset manuscripts
20 2023-01-21 17:47:38 : Opting for variant header at row 5.


2023-01-21 17:47:40,258 INFO Calling for resources with sheet_name Resources.


...

20 2023-01-21 17:50:31 : Loading dataset R0074_locations
20 2023-01-21 17:50:31 : Opting for variant header at row 5.


2023-01-21 17:50:33,469 INFO Using R0074_locations and id.
2023-01-21 17:50:33,469 INFO R0074_locations For columns id the suffix from _R0074 set was glued.
2023-01-21 17:50:33,470 INFO Using R0074_locations and locationrelation1_value_id.
2023-01-21 17:50:33,470 INFO R0074_locations For columns locationrelation1_value_id the suffix from _R0074 set was glued.
2023-01-21 17:50:33,471 INFO Using R0074_locations and locationrelation2_value_id.
2023-01-21 17:50:33,472 INFO R0074_locations For columns locationrelation2_value_id the suffix from _R0074 set was glued.
2023-01-21 17:50:33,472 INFO Using R0074_locations and superordinate_location_id.
2023-01-21 17:50:33,473 INFO R0074_locations For columns superordinate_location_id the suffix from _R0074 set was glued.
2023-01-21 17:50:33,473 INFO Using R0074_locations and identificate_id.
2023-01-21 17:50:33,474 INFO R0074_locations For columns identificate_id the suffix from _R0074 set was glued.
2023-01-21 17:50:33,477 INFO Calling for R0104_

20 2023-01-21 17:50:33 : Hyperlinks were detected and transformed in columns [].
20 2023-01-21 17:50:33 : Dropping empty columns in the dataset R0074_locations : (1000, 36)
20 2023-01-21 17:50:33 : Deleted 947 empty rows by label.
20 2023-01-21 17:50:33 : Loaded and prepared dataset R0074_locations : (53, 36)
20 2023-01-21 17:50:33 : Making pickle cache of  R0074_locations with separeted header file : (53, 36)
20 2023-01-21 17:50:34 : Loading dataset R0104_events
20 2023-01-21 17:50:34 : Opting for variant header at row 5.


2023-01-21 17:50:49,727 INFO Using R0104_events and id.
2023-01-21 17:50:49,728 INFO R0104_events For columns id the suffix from _R0104 set was glued.
2023-01-21 17:50:49,728 INFO Using R0104_events and participant_id.
2023-01-21 17:50:49,729 INFO R0104_events For columns participant_id the suffix from _R0104 set was glued.
2023-01-21 17:50:49,729 INFO Using R0104_events and inquisitor_id.
2023-01-21 17:50:49,730 INFO R0104_events For columns inquisitor_id the suffix from _R0104 set was glued.
2023-01-21 17:50:49,731 INFO Using R0104_events and notary_id.
2023-01-21 17:50:49,731 INFO R0104_events For columns notary_id the suffix from _R0104 set was glued.
2023-01-21 17:50:49,732 INFO Using R0104_events and witness_id.
2023-01-21 17:50:49,732 INFO R0104_events For columns witness_id the suffix from _R0104 set was glued.
2023-01-21 17:50:49,733 INFO Using R0104_events and spatial_ref_value_id.
2023-01-21 17:50:49,734 INFO R0104_events For columns spatial_ref_value_id the suffix from _R01

20 2023-01-21 17:50:49 : Hyperlinks were detected and transformed in columns [].
20 2023-01-21 17:50:49 : Dropping empty columns in the dataset R0104_events : (1622, 57)
20 2023-01-21 17:50:49 : Deleted 1562 empty rows by label.
20 2023-01-21 17:50:49 : Loaded and prepared dataset R0104_events : (60, 57)
20 2023-01-21 17:50:49 : Making pickle cache of  R0104_events with separeted header file : (60, 57)


In [286]:
# making empty table of values
tables['values'] = pd.DataFrame(columns=['id','value','origin'])
tables['locations'] = pd.DataFrame(columns=['id','value','origin','legacyId'])
tables['events'] = pd.DataFrame(columns=['id','value','origin','legacyId'])
tables['props'] = pd.DataFrame(columns=['id','type','type_id','value','value_id','original_field','origin','entityId','legacyId'])
tables['relations'] = pd.DataFrame(columns=['id','type','logic','certainty','entityIds','origin'])

In [187]:
tables['R0008_events']

,id,label,legacyId,label_language,entity_logical_type,resource_id,document_no,page_range_start,page_range_end,page_range_concatenated,...,marked_as_incriminated_label,negat_mentioned_in_margine,circumstance5_id,circumstance5_label,note,participant_sex,location_class,editor,done,checked
0,f0bc8874-7462-4f85-a514-85072e180fd0,[Rubrica],E0137_R0008,Latin,definite,R0002,,52,52,52,...,,,,,,,,David Zbíral,y,
1,3ee2f71b-5334-4498-8a53-208d19af8198,[Preambula],E0138_R0008,Latin,definite,R0002,,52,52,52,...,,,,,,,,David Zbíral,y,
2,eaf86656-6751-4d19-9770-709a1ed09e33,Abiuratio et obligatio Andree Saramite,E0001_R0008,Latin,definite,R0002,I.1,52,56,52-56,...,,,,,,male,camera officii inquisitionis,David Zbíral,y,
3,b52e81ea-1629-49a8-b7ee-e51536b43b09,Dictum suprascripti Andree,E0002_R0008,Latin,definite,R0002,I.2,56,60,56-60,...,,n,,,Acknowledges the guilt of Ricadona and Melior ...,male,camera officii inquisitionis,David Zbíral,y,
4,2d82ff2a-dea5-4469-8e60-e6e9826698af,Precepta facta suprascripto Andree,E0003_R0008,Latin,definite,R0002,I.3,60,60,60,...,,,,,,male,camera officii inquisitionis,David Zbíral,y,y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,f1112836-e0c8-4f99-ab13-e4ff3243068b,[Abiuratio Andree Saramite in manibus Mayfredi...,E0131_R0008,Latin,definite,R0002,II.26,176,176,176,...,,,,,,male,,David Zbíral,y,
136,6900af52-4b31-4004-9a8f-9db4cbf6b038,[Admonitio Francisci archiepiscopi],E0133_R0008,Latin,definite,R0002,III.1,202,202,202,...,,,,,,female,,David Zbíral,y,
137,25495c6e-e302-4a8a-ac9c-b4ffa5847bd5,[Dictum Carabelle de Toschanis],E0134_R0008,Latin,definite,R0002,III.19,248,248,248,...,,,,,"""si ipsa fuisset recordata de predictis, quand...",female,,David Zbíral,y,
138,334d7bd6-927f-4e37-ad23-f469a8dd3fd7,[Dictum Francischi de Garbagnate],E0135_R0008,Latin,definite,R0002,III.21,254,254,254,...,,,,,,male,,David Zbíral,y,


In [23]:
header_infos['R0082_events']

,id,label,label_language,entity_logical_type,resource_id,document_no,page_range_starts,page_range_ends,page_range_concatenated,class_id,...,accused_label,mentioned_id,mentioned_label,denied_id,denied_label,note,editor,done,checked,legacyId
0,,,,,,,,,,,...,,,,,,"Save as the multi-note (if ""#"" is present, sep...",audits,,,
1,,,,,,,,,id,id,...,,id,,id,,,,,,
2,,,,,,,,,R0034,Classification,...,,C2924,,C3403,,note,,,,
3,inside,inside,inside,inside,discard,discard,discard,discard,reference_part,relation,...,discard,propvalue,discard,propvalue,discard,inside,special,discard,discard,inside


In [341]:
# Save a dictionary into a pickle file.
import dill
pickle.dump( tables, open( "./data/import-input-tables-canonical.pickle", "wb" ) )
pickle.dump( header_infos, open( "./data/import-input-headers-canonical.pickle", "wb" ) )
pickle.dump( dh.entity_ids, open( "./data/import-input-entity_ids-canonical.pickle", "wb" ) )

In [ ]:
# load from pickle file
# tables = pickle.load( open( "./data/import-input-tables.pickle", "rb" ) )
# header_infos = pickle.load( open( "./data/import-input-headers.pickle", "rb" ) )
# entity_ids = pickle.load( open( "./data/import-input-entity_ids.pickle", "rb" ) )

In [12]:
entity_ids.keys()

dict_keys(['texts', 'manuscripts', 'resources', 'actions', 'concepts', 'persons', 'locations', 'events', 'groups'])

In [20]:
# manual corrections in the input data

### Responsible editors

#### Data
Only concepts have "editors" filled.

#### Specification

Only entities have audits, i.e. the prop object, the ref object don't. (Could be wrong?)

**Rationale:**  Each entity will have 2 audit creation records. One with the user "import", second with the user "responsible editor.

Implementaions possiblities:
  - make the audits during the parsing : the way
  - make the audits afer the parsing is done (for secondary created entities, I will not have the context, and will be problem to find a responsible editor
  -

In [287]:
# the names in the input tables  vs their user ids
# the users are imported through the user.json file

#David Zbíral	david.zbiral@mail.muni.cz
#Robert Shaw	robert.shaw@mail.muni.cz
#Jan Král	kral.jan@mail.muni.cz
#Reima Välimäki	reima.valimaki@gmail.com
#Lidia Hinz-Wieczorek	lidhin93@gmail.com
#Davor Salihović	davor.salihovic@mail.muni.cz
#Katia Riccardo	katia.riccardo@mail.muni.cz
#Larissa de Freitas Lyth	larissa.lyth@mail.muni.cz
#Larissa de Freitas Lyth

editors = {"import":"0", "David Zbíral":"100", "Robert Shaw":"101", "Davor Salihović":"102", "Katia Riccardo":"103", "Jan Král":"104", "Reima Välimäki":"105","Lidia Hinz-Wieczorek":"106", "Larissa de Freitas Lyth":"107"}

# Run encapsulated

In [9]:
# load clean input from pickle file
tables = pickle.load( open( "./data/import-input-tables.pickle", "rb" ) )
header_infos = pickle.load( open( "./data/import-input-headers.pickle", "rb" ) )
entity_ids = pickle.load( open( "./data/import-input-entity_ids.pickle", "rb" ) )

In [37]:
# dh contain input data, but also generated data (aka generated resource) and additional output (relatins, audits ...)
dh = inkimplib.DataHolder() # singleton
dh.pass_data(tables, header_infos, entity_ids, table_to_entity, editors)
dh.reset_generated_collection()

2023-01-13 20:05:56,446 INFO Start
2023-01-13 20:05:56,448 INFO TEXTS Uncertain parsing instructions in 0 columns: .
2023-01-13 20:05:56,469 INFO TEXTS 64 columns have been dropped (discard:64, unknown:0: []). Table now has 52 columns, inside:9,propvalue:14, special:5, proptype: 4, proptype_2nd: 0, dependent:0, reference_part:0. Originally 116 columns.
2023-01-13 20:05:56,471 INFO ACTIONS Uncertain parsing instructions in 0 columns: .
2023-01-13 20:05:56,496 INFO ACTIONS 63 columns have been dropped (discard:63, unknown:0: []). Table now has 25 columns, inside:14,propvalue:0, special:1, proptype: 0, proptype_2nd: 0, dependent:0, reference_part:0. Originally 88 columns.
2023-01-13 20:05:56,498 INFO MANUSCRIPTS Uncertain parsing instructions in 0 columns: .
2023-01-13 20:05:56,508 INFO MANUSCRIPTS 35 columns have been dropped (discard:35, unknown:0: []). Table now has 31 columns, inside:8,propvalue:8, special:4, proptype: 4, proptype_2nd: 0, dependent:0, reference_part:0. Originally 66 c

In [259]:
dh.reset_generated_collection()

In [260]:
logger.info(f"Start")
cp = inkimplib.ParseController(entity_list=['texts','actions', 'manuscripts','resources','concepts'])
cp.parse()
logger.info(f"End")

2023-01-21 16:47:46,684 INFO Start
2023-01-21 16:47:46,687 INFO TEXTS Uncertain parsing instructions in 0 columns: .
2023-01-21 16:47:46,707 INFO TEXTS 64 columns have been dropped (discard:64, unknown:0: []). Table now has 52 columns, inside:9,propvalue:14, special:5, proptype: 4, proptype_2nd: 0, dependent:0, reference_part:0. Originally 116 columns.
2023-01-21 16:47:46,710 INFO ACTIONS Uncertain parsing instructions in 0 columns: .
2023-01-21 16:47:46,733 INFO ACTIONS 63 columns have been dropped (discard:63, unknown:0: []). Table now has 25 columns, inside:14,propvalue:0, special:1, proptype: 0, proptype_2nd: 0, dependent:0, reference_part:2. Originally 88 columns.
2023-01-21 16:47:46,735 INFO MANUSCRIPTS Uncertain parsing instructions in 0 columns: .
2023-01-21 16:47:46,746 INFO MANUSCRIPTS 35 columns have been dropped (discard:35, unknown:0: []). Table now has 31 columns, inside:8,propvalue:8, special:4, proptype: 4, proptype_2nd: 0, dependent:0, reference_part:0. Originally 66 c

In [246]:
dh.tables['resources'][dh.tables['resources']['label'].str.contains("Douais")]

,id,label,legacyId,detail,label_language,for_migration_to_ddb1,do_before_migration,do_after_migration,migration_check_by_david_done,ready_for_migration,...,parsing_type,parsing_entity,dissinet_repository_url,spreadsheet_id,sheet_name,note,editor,parsing_rows_explained,dissinet_respository_url,origin
134,1afca8bb-aadb-478e-a346-b96be9ae05b2,"Douais, 1900",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.zotero.org/groups/446972/david_zbi...,"special C1188>:edition_1Douais, 1900|https://w..."
151,0f32b8b0-9295-494f-9d06-d28004b78804,"Douais, 1900",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.zotero.org/groups/446972/david_zbi...,"special C1188>:edition_2Douais, 1900|https://w..."
164,08e99be1-5227-49eb-aa4d-4e936e3cbd00,"Douais, 1900",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.zotero.org/groups/446972/david_zbi...,"special C1188>:edition_1Douais, 1900|https://w..."
165,90b15e22-bfd6-4c3f-b250-f819e5a41a8d,"Douais, 1900",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.zotero.org/groups/446972/david_zbi...,"special C1188>:edition_1Douais, 1900|https://w..."


In [165]:
# save specific state of the dh object
import dill as pickle
pickle.dump(dh, open("./data/dh.with.meta.canonical-17-1.pickle","wb"))
# saved also ./data/dh.with.meta.canonical.pickle, 13.1. 2022 - theoretically definitive meta entities (texts, actions, manuscripts, resources, concepts) import
# saved also ./data/dh.with.meta.canonical-17-1.pickle, 17.1. 2022 - theoretically definitive meta entities (texts, actions, manuscripts, resources, concepts) import, with correct input case area datasets

In [332]:
# load specific state of the dh object
import dill as pickle
dh = pickle.load(open( "./data/dh.with.meta.canonical-17-1.pickle", "rb" ))    # contains errors in bologna ids :(

In [333]:
for key, table in tables.items():
  rewrite = ['texts','resources','actions','conpcets', 'manuscripts']

  if "_" in key or key in rewrite:
    dh.tables[key] = tables[key]
    dh.header_infos[key] = header_infos[key]

In [334]:
dh.checkIDs(fix_difference=True)

############## texts
Data group: texts
############## texts 0.
############## manuscripts
Data group: manuscripts
############## manuscripts 0.
############## resources
Data group: resources
############## resources 0.
############## actions
Data group: actions
############## actions 0.
############## concepts
Data group: concepts
############## concepts 0.
############## R0006_persons
Data group: persons
############## R0006_persons 0.
############## R0007_locations
Data group: locations
############## R0007_locations 0.
############## R0008_events
Data group: events
############## R0008_events 0.
############## R0075_persons
Data group: persons
############## R0075_persons 0.
############## R0035_locations
Data group: locations
############## R0035_locations 0.
############## R0083_events
Data group: events
############## R0083_events 0.
############## R0105_groups
Data group: groups
############## R0105_groups 0.
############## R0030_persons
Data group: persons
############## R0030_

In [335]:
dh.reset_generated_collection()

In [339]:
logger.info(f"Start")
# Guglielmites
# cp = inkimplib.ParseController(entity_list=[ 'R0006_persons', 'R0007_locations', 'R0008_events']) # ,'R0008_events'
# 'R0006_persons',

# Castellario
# cp = inkimplib.ParseController(entity_list=['R0075_persons', 'R0105_groups', 'R0035_locations','R0083_events'])

# Bologna
# cp = inkimplib.ParseController(entity_list=['R0030_persons', 'R0087_groups', 'R0031_locations','R0082_events'])

# Carcassone # R0040_locations, R0039_persons
# cp = inkimplib.ParseController(entity_list=['R0039_persons', 'R0040_locations', 'R0079_events'])

#Casasco
# cp = inkimplib.ParseController(entity_list=['R0076_persons', 'R0074_locations','R0104_events'])

# all
# withou Bologna, 'R0030_persons', 'R0087_groups', 'R0031_locations','R0082_events',
# cp = inkimplib.ParseController(entity_list=['R0006_persons', 'R0007_locations','R0008_events', 'R0075_persons', 'R0105_groups', 'R0035_locations','R0083_events',  'R0039_persons', 'R0040_locations', 'R0079_events','R0076_persons', 'R0074_locations','R0104_events'])

cp = inkimplib.ParseController(entity_list=['texts','actions', 'manuscripts','resources','concepts','R0006_persons', 'R0007_locations','R0008_events', 'R0075_persons', 'R0105_groups', 'R0035_locations','R0083_events',  'R0039_persons', 'R0040_locations', 'R0079_events','R0076_persons', 'R0074_locations','R0104_events','R0030_persons', 'R0087_groups', 'R0031_locations','R0082_events'], dh=dh)

cp.parse()
logger.info(f"End")

2023-01-21 18:58:21,367 INFO Class IActant available.
2023-01-21 18:58:21,368 INFO Class IAction available.
2023-01-21 18:58:21,369 INFO Class IAudit available.
2023-01-21 18:58:21,370 INFO Class IConcept available.
2023-01-21 18:58:21,372 INFO Class IEntity available.
2023-01-21 18:58:21,374 INFO Class IEvent available.
2023-01-21 18:58:21,376 INFO Class IGroup available.
2023-01-21 18:58:21,376 INFO Class ILabel available.
2023-01-21 18:58:21,377 INFO Class ILocation available.
2023-01-21 18:58:21,379 INFO Class IObject available.
2023-01-21 18:58:21,380 INFO Class IPerson available.
2023-01-21 18:58:21,381 INFO Class IProp available.
2023-01-21 18:58:21,383 INFO Class IReference available.
2023-01-21 18:58:21,385 INFO Class IResource available.
2023-01-21 18:58:21,386 INFO Class IStatement available.
2023-01-21 18:58:21,387 INFO Class ITerritory available.
2023-01-21 18:58:21,388 INFO Class IUser available.
2023-01-21 18:58:21,389 INFO Class IValue available.
2023-01-21 18:58:21,389

In [340]:
# save to resulted dh
pickle.dump(dh, open("./data/dh.import-fullhck8-brackets.pickle","wb"))

In [273]:
pd.DataFrame(dh.additional_entities)

,class,id,label,language,detail,data,props,notes,status,references,legacyId
0,E,3adb9104-eef7-4525-bcf2-938c4bef0ebc,Creation of ‘Acta Concilii Tolosani’,eng,,{'logicalType': '1'},[],[Import batch [January2023] 2023-01-21 16:47:1...,1,[],E0001_T153
1,E,661bf3fc-1765-4014-9ec1-203ee0fb4198,Creation of ‘Acta Sancti Officii Bononie ab an...,eng,,{'logicalType': '1'},"[{'bundleEnd': False, 'bundleStart': False, 'c...",[Import batch [January2023] 2023-01-21 16:47:1...,1,[],E0002_T77
2,V,d3a32c8b-1e1a-4520-8f58-2b457054c72a,1291,,,{'logicalType': '1'},[],[Import batch [January2023] 2023-01-21 16:47:1...,1,[],NaN
3,V,cd39408e-cff2-4aa2-a363-3eaacd4190da,1291,,,{'logicalType': '1'},[],[Import batch [January2023] 2023-01-21 16:47:1...,1,[],NaN
4,V,cff0adc7-aed4-4d28-8035-10fb582c5b23,1310,,,{'logicalType': '1'},[],[Import batch [January2023] 2023-01-21 16:47:1...,1,[],NaN
...,...,...,...,...,...,...,...,...,...,...,...
18851,T,1d28eaf4-077d-49b8-afa3-7d8d017e9f35,"Sentenza assolutoria di Benvenutus, rector S.P...",ita,,{'parent': {'territoryId': '6870cab5-fb77-45c9...,"[{'bundleEnd': False, 'bundleStart': False, 'c...",[Import batch [January2023] 2023-01-21 16:47:1...,1,[],T77-19-921
18852,V,929cada7-eb68-4668-9a78-ab2135c6c315,1309-10-16,,,{'logicalType': '1'},[],[Import batch [January2023] 2023-01-21 16:47:1...,1,[],NaN
18853,V,e61cd3e2-4f3f-4c8e-8ea9-f37f76183a2b,713-714,,,{'logicalType': '1'},[],[Import batch [January2023] 2023-01-21 16:47:1...,1,[],NaN
18854,T,f3277f80-a402-41d9-a5ff-d9e4b93e8d6a,Sentenza assolutoria di Petrus Mutinensis,ita,,{'parent': {'territoryId': '6870cab5-fb77-45c9...,"[{'bundleEnd': False, 'bundleStart': False, 'c...",[Import batch [January2023] 2023-01-21 16:47:1...,1,[],T77-19-922


In [ ]:
# cp = ParseController(entity_list=['texts','actions', 'manuscripts','resources','concepts','R0006_persons', 'R0007_locations','R0008_events'])
# cp = ParseController(entity_list=['texts','actions', 'manuscripts','resources','concepts','R0006_persons', 'R0007_locations','R0008_events',"R0075_persons","R0035_locations","R0083_events"])

# cp = ParseController(entity_list=['texts','actions', 'manuscripts','resources','concepts','R0006_persons', 'R0007_locations','R0008_events',"R0075_persons","R0035_locations","R0083_events"], existing_parser=cp, reparse_entity_list=['R0035_locations'])

# cp = ParseController(entity_list=['texts', "concepts", "R0008_events"])
#cp = ParseController(entity_list=[ "R0007_locations"])
# , 'R0006_persons', 'R0007_locations'
# cp = ParseController(entity_list=['texts','manuscripts','resources','concepts','actions','R0006_persons', 'R0007_locations']) # , 'R0006_person \as', 'R0007_locations'
# cp = ParseController(entity_list=['R0006_persons', 'R0007_locations'])
#cp = ParseController(entity_list=['R0008_events'])
#cp = ParseController(entity_list=['actions','concepts'])

# After parsing

### Merging reciprocal relation records
Antonym, PropertyReciprocal, SubjectActant1Reciprocal, Synonym, Related

In [342]:
# ANT, PRR, SAR, SYN, REL
def getE(eIds):
    eids = eIds.copy() # has to be copy, otherwise it modifies original relation_records list-arrays
    eids.sort()
    return eids[0]+"!"+eids[1]

rdf = pd.DataFrame(dh.relation_records)
rdf['eids'] = rdf['entityIds'].apply(lambda row:getE(row))
removeRR  = rdf[rdf[['type','eids']].duplicated()].sort_values('eids')['id'].to_list()
removeRR

['d3ded3bc-8c2c-46b3-8fbc-df81ef77fa44',
 'f9e1e0db-3694-4595-a3eb-0547f332e462',
 '0472c139-cba3-4e96-b5db-3c7307d97c37',
 'b4fe4134-9680-4ef6-8ccf-95fc49321966',
 'd421f257-2fa5-42d2-8fe1-7951926553d8',
 '994f195c-09f3-4464-a156-e517728e2eba',
 '525d3cbd-1f81-47bb-b899-e4f48d0237bd',
 '705d2c58-8e43-4da7-adf7-14c05df51e8b',
 '37ca8390-19fd-46f3-ad40-f40db2bbc5a6',
 'af8afc6e-cf7a-4d2e-84d4-4da51d97caf1',
 '8c5d26e8-34a3-4888-b0e6-65bd7ab43c1f',
 'b80cdb4d-e1c7-45f0-a767-4e4aa0c7c02a',
 '4619824d-6686-4a61-8fae-701f8427c5e8',
 '591e6767-304f-4c7d-a065-a990278fd4a2',
 '0ec17312-bf2d-4267-8962-a71053991693',
 '4a23cc19-58ef-4aef-be0f-7c6d48c9227c',
 '7957dc5d-a2dd-4ce1-a7fb-a6db8bfdc709',
 '01f96a2f-219b-4729-8254-296c68dd8013',
 'b7cf86e7-ca29-44b0-85af-f8003401625d',
 '0e483978-e389-4cfd-b9b7-2a2a03062d7d',
 'b3bdfab8-c83b-4795-a317-d178086c1bcd',
 '7a8a0f14-0316-4531-ab76-fdd39ac0c268',
 '5aba8ee1-a6e3-4ddb-9d38-a24851803b1a',
 '78e7210a-e0a3-49fe-9bbb-53712fb2bfd8',
 'd406abbb-1080-

In [343]:
relation_records_cleaned = []
for item in dh.relation_records:
    if item['id'] not in removeRR:
        relation_records_cleaned.append(item)

In [344]:
len(dh.relation_records), len(relation_records_cleaned)

(15028, 14588)

In [40]:
# check for relations entityIds
# a=0
# for item in relation_records_cleaned:
#     #if "81e38bb2-6f84-4b30-acac-b6931929dbaf" == item['id']:
#     if "7db804db-fa2d-4624-810d-b3440e2f4001" in item['entityIds'][0] and item['type']=="SCL":
#         a += 1
#         print(item)
# print(a)

{'id': 'dbb07e20-a1c6-49a0-a47e-dfde606f8e88', 'type': 'SCL', 'entityIds': ['7db804db-fa2d-4624-810d-b3440e2f4001', '479cc4f3-b877-44a5-a0dd-c6a7cc813163']}
1


In [347]:
# test

dh.tables['resources'][dh.tables['resources']['label'].str.contains("Paris")]

,id,label,legacyId,detail,label_language,for_migration_to_ddb1,do_before_migration,do_after_migration,migration_check_by_david_done,ready_for_migration,...,component_label,parsing_type,parsing_entity,dissinet_repository_url,spreadsheet_id,sheet_name,note,editor,parsing_rows_explained,origin
111,377a13ab-67ce-4d34-b443-c92c589f9058,"Paul de Saint-Père de Chartres, ""Monasterii Sa...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,special C1188>:edition_1Paul de Saint-Père de ...
113,795bdae8-27d7-4c32-a756-a3b6bad138ae,"Adémar de Chabannes, Chronique, ed. J. Chavano...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,"special C1188>:edition_2Adémar de Chabannes, C..."
125,82d3115b-dba9-4734-be55-2cdbb8523ec8,Johannes monachus ad Olibam in: André de Fleur...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,special C1188>:edition_1Johannes monachus ad O...
164,3fc68e8e-9c57-402a-8f8c-eaec734de6ef,"André de Fleury, ""Vie de Gauzlin, abbé de Fleu...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,"special C1188>:edition_1André de Fleury, ""Vie ..."
167,ccb120c6-7687-4a93-b6d7-17c30c33fd27,"Reproduction of Paris, Bibliothèque nationale ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,https://gallica.bnf.fr/ark:/12148/btv1b10035999g,NaN,NaN,NaN,NaN,NaN,Making Resource entity 'link|https://gallica.b...
168,c38802f8-6682-4726-b3a4-0bafddd4b055,"Reproduction of Paris, Bibliothèque nationale ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,https://gallica.bnf.fr/ark:/12148/btv1b10036000n,NaN,NaN,NaN,NaN,NaN,Making Resource entity 'link|https://gallica.b...
169,0a193b1a-e497-4318-b3f7-ba352dcf3a70,"Reproduction of Paris, Bibliothèque nationale ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,https://gallica.bnf.fr/ark:/12148/btv1b100365190,NaN,NaN,NaN,NaN,NaN,Making Resource entity 'link|https://gallica.b...
170,6dfb881c-1449-48d3-b134-4a441fb486a9,"Reproduction of Paris, Bibliothèque nationale ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,https://gallica.bnf.fr/ark:/12148/btv1b100361092,NaN,NaN,NaN,NaN,NaN,Making Resource entity 'link|https://gallica.b...
172,062ea1e6-b079-41f8-bde4-9428128c86c7,"Reproduction of Paris, Bibliothèque nationale ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,https://gallica.bnf.fr/ark:/12148/btv1b100338354,NaN,NaN,NaN,NaN,NaN,Making Resource entity 'link|https://gallica.b...
174,5ba1425b-45a1-4c10-8336-00065cedf14e,"Reproduction of Paris, Bibliothèque nationale ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,https://gallica.bnf.fr/ark:/12148/btv1b5315700...,NaN,NaN,NaN,NaN,NaN,Making Resource entity 'link|https://gallica.b...


In [315]:
dh.entity_ids['resources']['M21']

'3feae15a-e17a-4844-b6a3-88678a4720db'

In [316]:
dh.checkIDs()

############## texts
Data group: texts
############## texts 0.
############## manuscripts
Data group: manuscripts
############## manuscripts 0.
############## resources
There are duplicates! 107    NaN
108    NaN
109    NaN
110    NaN
111    NaN
      ... 
180    NaN
181    NaN
182    NaN
183    NaN
184    NaN
Name: legacyId, Length: 78, dtype: object
Data group: resources
############## resources 0.
############## actions
Data group: actions
############## actions 0.
############## concepts
Data group: concepts
############## concepts 0.
############## R0006_persons
Data group: persons
############## R0006_persons 0.
############## R0007_locations
Data group: locations
############## R0007_locations 0.
############## R0008_events
Data group: events
############## R0008_events 0.
############## R0075_persons
Data group: persons
############## R0075_persons 0.
############## R0035_locations
Data group: locations
############## R0035_locations 0.
############## R0083_events
Data group: e

### Profiling

In [ ]:
#%reload_ext line_profiler

# def profile():
#    cp.parsers['concepts'].parse_rows()

#%lprun -f EntityMapper.get_entity_id profile()

In [ ]:
# %prun -s tottime profile()

In [ ]:
# %lprun?

### Last checks

In [345]:
logger.info(f"There are {len(dh.additional_entities)} additionally created entities (e.g. values, resources ...).")

2023-01-21 19:21:55,069 INFO There are 18856 additionally created entities (e.g. values, resources ...).


# Output the parsed data

In [299]:
# root data
root_territory = """
 {
    "id": "T0",
    "class": "T",
    "data": { "parent": false },
    "label": "root",
    "detail": "",
    "language": "lat",
    "notes": [],
    "props": []
  }
"""

In [355]:
# open robert's example of Nior coding
robert_data1317 = {
  "entities": json.load(open("../datasets/experiments/diff1317/entities.json","r")),
  "relations": json.load(open("../datasets/experiments/diff1317/relations.json","r")),
  "audits": json.load(open("../datasets/experiments/diff1317/audits.json","r"))
}

robert_data1719 = {
  "entities": json.load(open("../datasets/experiments/diff1719/entities.json","r")),
  "relations": json.load(open("../datasets/experiments/diff1719/relations.json","r")),
  "audits": json.load(open("../datasets/experiments/diff1719/audits.json","r"))
}

robert_data1920 = {
  "entities": json.load(open("../datasets/experiments/diff1920/entities.json","r")),
  "relations": json.load(open("../datasets/experiments/diff1920/relations.json","r")),
  "audits": json.load(open("../datasets/experiments/diff1920/audits.json","r"))
}

robert_data2021 = {
  "entities": json.load(open("../datasets/experiments/diff2021/entities.json","r")),
  "relations": json.load(open("../datasets/experiments/diff2021/relations.json","r")),
  "audits": json.load(open("../datasets/experiments/diff2021/audits.json","r"))
}


r_entities1317 = json.dumps(robert_data1317["entities"])[1:-1]
r_entities1719 = json.dumps(robert_data1719["entities"])[1:-1]
r_entities1920 = json.dumps(robert_data1920["entities"])[1:-1]
r_entities2021 = json.dumps(robert_data2021["entities"])[1:-1]

r_relations1317 = json.dumps(robert_data1317['relations'])[1:-1]
r_relations1719 = json.dumps(robert_data1719['relations'])[1:-1]
r_relations1920 = json.dumps(robert_data1920['relations'])[1:-1]
r_relations2021 = json.dumps(robert_data2021['relations'])[1:-1]
#r_audits = json.dumps(robert_data['audits'])[1:len(str(robert_data['audits']))-1]

In [356]:
# ideal "2023-01-13T20:05:56Z"
import datetime
for item in robert_data1317["audits"]:
  #print(item['date']['epoch_time']), print(datetime.datetime.fromtimestamp(item['date']['epoch_time']))
  new_time_chunks = str(datetime.datetime.fromtimestamp(item['date']['epoch_time'])).split(" ")
  new_time = new_time_chunks[0]+"T"+new_time_chunks[1].split(".")[0]+"Z"
  item['date'] = new_time

r_audits1317 = json.dumps(robert_data1317['audits'])[1:-1]

for item in robert_data1719["audits"]:
  #print(item['date']['epoch_time']), print(datetime.datetime.fromtimestamp(item['date']['epoch_time']))
  new_time_chunks = str(datetime.datetime.fromtimestamp(item['date']['epoch_time'])).split(" ")
  new_time = new_time_chunks[0]+"T"+new_time_chunks[1].split(".")[0]+"Z"
  item['date'] = new_time

r_audits1719 = json.dumps(robert_data1719['audits'])[1:-1]

for item in robert_data1920["audits"]:
  #print(item['date']['epoch_time']), print(datetime.datetime.fromtimestamp(item['date']['epoch_time']))
  new_time_chunks = str(datetime.datetime.fromtimestamp(item['date']['epoch_time'])).split(" ")
  new_time = new_time_chunks[0]+"T"+new_time_chunks[1].split(".")[0]+"Z"
  item['date'] = new_time

r_audits1920 = json.dumps(robert_data1920['audits'])[1:-1]

for item in robert_data2021["audits"]:
  #print(item['date']['epoch_time']), print(datetime.datetime.fromtimestamp(item['date']['epoch_time']))
  new_time_chunks = str(datetime.datetime.fromtimestamp(item['date']['epoch_time'])).split(" ")
  new_time = new_time_chunks[0]+"T"+new_time_chunks[1].split(".")[0]+"Z"
  item['date'] = new_time

r_audits2021 = json.dumps(robert_data2021['audits'])[1:-1]

In [357]:
len(r_entities1317), len(r_entities1719),len(r_entities1920),len(r_entities2021)

(35168, 54585, 31748, 7110)

In [490]:
r_entities = r_entities1317 + "," + r_entities1719 + "," + r_entities1920 + "," + r_entities2021
r_relations = r_relations1317 + "," + r_relations1719 + "," + r_relations1920 + "," + r_relations2021
r_audits = r_audits1317 + "," + r_audits1719 + "," + r_audits1920 + "," + r_audits2021

In [492]:

# save json object list of the parsed entities from google sheets in the text file
with open('../datasets/all-test/new_entities.json', 'w', encoding='utf-8') as f:
    #f.write(str(cp.parsers['texts'].js_objects))
    json.dump(dh.js_objects, f)

# save json object list of the "newly created entities" in the text file
with open('../datasets/all-test/additional_entities.json', 'w', encoding='utf-8') as f:
    #f.write(str(cp.parsers['texts'].js_objects))
    json.dump(dh.additional_entities, f)

# read  entities.json
with open('../datasets/all/entities.json','r') as f:
    #entities_content = f.readlines()
    entities_content = f.read().replace('\n', '')

# read  entities.json
with open('../datasets/all-test/new_entities.json','r') as f:
    #entities_content = f.readlines()
    new_entities_content = f.read().replace('\n', '')

additional_entities_string = json.dumps(dh.additional_entities)

# write new and combine with old test data
with open('../datasets/all-test/entities.json','w', encoding='utf-8') as f:
    #merge_content = entities_content[0:-1] +  str(cp.parsers['texts'].js_objects)[1:]
    merge_content = entities_content[0:-1] +", " + additional_entities_string[1:-1]+ ", " + new_entities_content[1:]
    f.write(str(merge_content))

# write just the new parse data to the entities json.
with open('../datasets/all-parsed/entities.json','w', encoding='utf-8') as f:
    #merge_content = entities_content[0:-1] +  str(cp.parsers['texts'].js_objects)[1:]
    merge_content =  "[" + root_territory + "," + additional_entities_string[1:-1]+ ","+ r_entities +","+  new_entities_content[1:]
    f.write(str(merge_content))

relations_entities_string = json.dumps(relation_records_cleaned)
audits_entities_string = json.dumps(dh.audits)

# write relations
with open('../datasets/all-parsed/relations.json','w', encoding='utf-8') as f:
    merge_content =  relations_entities_string[0:-1] + "," + r_relations + "]"
    f.write(str(merge_content))
    #json.dump(relation_records_cleaned, f)

# write the audits json.
with open('../datasets/all-parsed/audits.json','w', encoding='utf-8') as f:
    merge_content =  audits_entities_string[0:-1] +  ","+ r_audits + "]"
    f.write(str(merge_content))
    #json.dump(dh.audits, f)

# write just the new parse data to the entities json.
with open('../datasets/all-parsed-wr/entities.json','w', encoding='utf-8') as f:
    #merge_content = entities_content[0:-1] +  str(cp.parsers['texts'].js_objects)[1:]
    merge_content =  "[" + root_territory + "," + additional_entities_string[1:-1] +","+  new_entities_content[1:]
    f.write(str(merge_content))


# write relations
with open('../datasets/all-parsed-wr/relations.json','w', encoding='utf-8') as f:
    merge_content =  relations_entities_string[0:-1] + "]"
    f.write(str(merge_content))
    #json.dump(relation_records_cleaned, f)

# write the audits json.
with open('../datasets/all-parsed-wr/audits.json','w', encoding='utf-8') as f:
    merge_content =  audits_entities_string[0:-1] + "]"
    f.write(str(merge_content))


logger.info("END")

2023-01-22 10:19:38,084 INFO END


In [493]:
# simple 2 strategy

parsed_e_df = pd.DataFrame(json.load(open("../datasets/all-parsed-wr/entities.json","r"))).fillna("").set_index("id")
parsed_r_df = pd.DataFrame(json.load(open("../datasets/all-parsed-wr/relations.json","r"))).fillna("").set_index("id")
parsed_a_df = pd.DataFrame(json.load(open("../datasets/all-parsed-wr/audits.json","r"))).fillna("").set_index("id")

ddb1_e_df = pd.DataFrame(json.load(open("../datasets/experiments/21/entities.json","r"))).fillna("").set_index("id")
ddb1_r_df = pd.DataFrame(json.load(open("../datasets/experiments/21/relations.json","r"))).fillna("").set_index("id")
ddb1_a_df = pd.DataFrame(json.load(open("../datasets/experiments/21/audits.json","r"))).fillna("").set_index("id")



In [447]:
len(parsed_e_df), len(ddb1_e_df)

(29360, 27980)

In [468]:
ddb1_e_df['notes_s'] = ddb1_e_df['notes'].astype(str)

In [469]:
# robert_e_df = ddb1_e_df[(~ddb1_e_df.index.isin(parsed_e_df.index)) & (ddb1_e_df['notes'].str.contains("Import"))]
robert_e_df = ddb1_e_df[~ddb1_e_df['notes_s'].str.contains("Import")]
robert_e_df

,class,data,detail,label,language,legacyId,notes,props,references,status,isTemplate,notes_s
id,,,,,,,,,,,,
025d451c-0fce-436c-8d37-0d13ac7f90a9,C,{},supporter of heretics (i.e. Cathars),fautor hereticorum,lat,,[],"[{'bundleEnd': False, 'bundleOperator': 'a', '...",[],4,False,[]
0a2ba32a-4c7e-40e4-a684-a65e6044e3c2,E,{'logicalType': '1'},,metus huius inquisitionis,lat,,[],[],[],1,False,[]
0e8166b4-f424-4717-b29e-685899e1b3c3,A,"{'entities': {'a1': ['P', 'G'], 'a2': ['C', 'E...",bore witness (in front of sb - about st),testatus/a,lat,,[],[],[],1,False,[]
0fcf21d7-da3b-4fb4-bebe-bd5930685b12,V,{},,34v,lat,,[],[],[],1,False,[]
10d2538b-d7d7-4a4f-9e34-1f7993337dc9,V,{},,34r,lat,,[],[],[],1,False,[]
...,...,...,...,...,...,...,...,...,...,...,...,...
f69921f2-5f70-4fd6-84da-16dba150656b,V,{},,34v,lat,,[],[],[],1,False,[]
f71126df-fc8f-42d0-977d-4f3414682bee,C,{},reporting,relatio,lat,,[],[],[],4,False,[]
f924fdff-4370-4b6d-a197-f2b289653117,V,{},,36r,lat,,[],[],[],1,False,[]


In [470]:
robert_r_df = ddb1_r_df[~ddb1_r_df.index.isin(parsed_r_df.index)]
robert_r_df

,entityIds,order,type,certainty
id,,,,
0002ea21-f57a-4066-95e2-3f5cc17adf45,"[6832b685-7391-4e6f-9dee-53ab048bced4, 390ac71...",10.0,A2S,
00055fff-5cbf-4e4f-a5b9-e9f8cd5d3b25,"[d08daee5-e2b4-458e-af9a-5d25942cee21, b1f631b...",10.0,REL,
00062934-e2cc-45e0-a4c2-f3efe0a790ee,"[9238a54a-8b50-4418-8a8a-5d8e1aeed77b, eabcf67...",1.0,CLA,
00079b82-f3eb-4815-b4f6-2a40ac887484,"[e7d8ae89-792a-4586-b9e2-4d83e3ea8824, adc03f9...",10.0,CLA,
00099734-9287-430a-9e50-5a98dcd4a6fc,"[469ca1dd-734b-4534-9bfc-fdaae17dc953, cbf3b28...",1.0,CLA,
...,...,...,...,...
ffea7bb7-15aa-4a61-972e-ba40fa3dea12,"[2abe4053-4018-4275-b20a-2e1021ac8d0c, 0d8e22d...",10.0,SOL,
ffef934d-3ddd-4382-8c4e-38430e3df341,"[4a4b7303-7823-43cc-8d85-546db731e3e5, 4762d66...",10.0,SOL,
fff1ddfd-e0c3-425a-856b-ec42cccacbc6,"[b2c92fdf-dc0d-4699-ac58-27c3472f52c3, 5904cba...",10.0,CLA,


In [ ]:
#  after stitching investigation


In [494]:
#  after stitching

# - R entita  s label "Reproduction of Paris, Bibliothèque nationale de France, fonds Doat, vol. 21"  musí mít guid 35a13dc0-783a-466d-9850-71c50b1f528d

with open('../datasets/all-parsed/entities.json','r', encoding='utf-8') as f:
  econtent = f.read()
  econtent = econtent.replace("ccb120c6-7687-4a93-b6d7-17c30c33fd27","35a13dc0-783a-466d-9850-71c50b1f528d")

with open('../datasets/all-parsed/entities.json','w', encoding='utf-8') as f:
  f.write(econtent)

In [495]:
# take statement content from the last downloaded DDB data
with open('../datasets/experiments/21/entities.json','r', encoding='utf-8') as f:
  # ddb1e = f.read()
  ddb1e = json.load(f)
  ddb1e_df = pd.DataFrame(ddb1e).set_index("id")

with open('../datasets/all-parsed/entities.json','r', encoding='utf-8') as f:
  # ddb1e = f.read()
  parsed_e = json.load(f)
  parsed_e_df = pd.DataFrame(parsed_e).set_index("id")


In [496]:
parsed_e_dfDD = parsed_e_df[~parsed_e_df.index.duplicated(keep='first')]

In [497]:
# reaplace all statements from ddb1e

parsed_e_dfDDnoS = parsed_e_dfDD[parsed_e_dfDD['class']!="S"]

In [498]:
ddb1s = ddb1e_df[ddb1e_df['class']=="S"]
result_e_df = pd.concat([parsed_e_dfDDnoS, ddb1s])
# parsed_e_df[parsed_e_df['class']=="S"]

In [499]:
result_e_df.reset_index().to_json('../datasets/stitched/entities.json',orient="records")

In [388]:
x = 0
for item in parsed_e_df[parsed_e_df['class']=="S"].index:
  # print(item)
  x += 1
  print(x, item)
  if item in ddb1e_df.index:
    print('*',item)

1 38e5b435-bb22-4b09-97be-7acdae65e370
* 38e5b435-bb22-4b09-97be-7acdae65e370
2 795a5729-1052-48c9-ae9a-d89a63d4c4c7
* 795a5729-1052-48c9-ae9a-d89a63d4c4c7
3 7aa92591-58cf-41c5-a4fc-d894914bedf8
* 7aa92591-58cf-41c5-a4fc-d894914bedf8
4 82bbf78f-7995-4928-9e49-de29a46d0b05
* 82bbf78f-7995-4928-9e49-de29a46d0b05
5 8338273a-151d-44a8-94ba-0ff6ac69f5db
* 8338273a-151d-44a8-94ba-0ff6ac69f5db
6 8ad01491-fff6-4762-81c2-a44316a2e32b
* 8ad01491-fff6-4762-81c2-a44316a2e32b
7 953cc66a-49e8-4386-8178-32d7aea5539f
* 953cc66a-49e8-4386-8178-32d7aea5539f
8 c3d073d2-8b30-49b1-9a46-e1260329924f
* c3d073d2-8b30-49b1-9a46-e1260329924f
9 1361864f-1682-4251-a832-c30da8216830
* 1361864f-1682-4251-a832-c30da8216830
10 1d271952-dbb0-4bf4-96e4-d20970f8dd66
* 1d271952-dbb0-4bf4-96e4-d20970f8dd66
11 29574a5a-2bd6-472e-ab43-e8507249c203
* 29574a5a-2bd6-472e-ab43-e8507249c203
12 3071ab0b-7554-43dd-b499-430e99d93fd3
* 3071ab0b-7554-43dd-b499-430e99d93fd3
13 3e51a831-2774-4262-98a4-b693aab89af8
* 3e51a831-2774-4262-

In [407]:
working = list(parsed_e_df.index)

In [409]:
len(set([x for x in working if working.count(x) > 1]))

47

In [396]:
len(working)

43

In [397]:
len(set(working))

34

In [155]:
len(dh.relation_records), len(relation_records_cleaned)

(15026, 14588)

In [19]:
dh.additional_entities


[{'class': 'E',
  'id': 'fe6753df-aeda-4d6c-8456-fabaf5ee1ab0',
  'label': 'Creation of ‘Acta Concilii Tolosani’',
  'language': '',
  'detail': '',
  'data': {'logicalType': '1'},
  'props': [],
  'notes': ['Import batch [development] 2023-01-13 19:35:45.939035',
   'special T153>:creation_event_id E0001_T153'],
  'status': '1',
  'references': [],
  'legacyId': 'E0001_T153'},
 {'class': 'E',
  'id': 'c7f31835-13a4-405c-a7aa-e630b40e14d8',
  'label': 'Creation of ‘Acta Sancti Officii Bononie ab anno 1291 usque ad annum 1310’',
  'language': '',
  'detail': '',
  'data': {'logicalType': '1'},
  'props': [],
  'notes': ['Import batch [development] 2023-01-13 19:35:45.939035',
   'special T77>:creation_event_id E0002_T77'],
  'status': '1',
  'references': [],
  'legacyId': 'E0002_T77'},
 {'class': 'V',
  'id': '09747a19-bf4b-48d1-b782-140bd1a2e8f6',
  'label': '1291',
  'language': '',
  'detail': '',
  'data': {'logicalType': '1'},
  'props': [],
  'notes': ['Import batch [development]

In [ ]:
# Log tables google sheet
# https://docs.google.com/spreadsheets/d/1UpbJckLrRYTCz7wcEYhhcc-ihJxiUENx1unCCQqE4TE/edit#gid=0


# d.open_gsheet("logtables","https://docs.google.com/spreadsheets/d/1UpbJckLrRYTCz7wcEYhhcc-ihJxiUENx1unCCQqE4TE")

# d.write_df_to_gsheet("logtables", "values", tables["values"])
# d.write_df_to_gsheet("logtables", "resources", tables["resources"])
# d.write_df_to_gsheet("logtables", "locations", tables["locations"])
# d.write_df_to_gsheet("logtables", "events", tables["events"])
# d.write_df_to_gsheet("logtables", "props", tables["props"])



In [156]:
d.open_gsheet("discouraged","https://docs.google.com/spreadsheets/d/10x8TV2OmkcGt6zHw-3B-I3BhonETa0iQV424vCB9qSQ/edit#gid=0")

20 2023-01-17 14:07:00 : Opening gsheet handle discouraged


In [157]:
discouraged_concepts = dh.tables['concepts'][tables['concepts']['status']=="discouraged"][['legacyId','label']]
discouraged_concepts_ids_list = discouraged_concepts['legacyId'].to_list()
discouraged_concepts_ids_list

['C2090',
 'C0408',
 'C0410',
 'C3069',
 'C0602',
 'C0165',
 'C0394',
 'C0541',
 'C0653',
 'C0744',
 'C0395',
 'C0417',
 'C2776',
 'C0125',
 'C0231',
 'C0611',
 'C0612',
 'C0683',
 'C0764',
 'C0784',
 'C0433',
 'C0636',
 'C0701',
 'C1271',
 'C1272',
 'C1273',
 'C0449',
 'C0455',
 'C0457',
 'C0531',
 'C0475',
 'C0672',
 'C0432',
 'C0439',
 'C0441',
 'C0673',
 'C0494',
 'C0495',
 'C0205',
 'C0206',
 'C0207',
 'C0498',
 'C0505',
 'C0518',
 'C0519',
 'C0520',
 'C2087',
 'C0203',
 'C0496',
 'C0497',
 'C0502',
 'C0503',
 'C0421',
 'C0422',
 'C0174',
 'C0176',
 'C0183',
 'C0184',
 'C0175',
 'C0177',
 'C2091',
 'C0210',
 'C0499',
 'C0500',
 'C1112',
 'C0628',
 'C0630',
 'C0180',
 'C0219',
 'C0525',
 'C0529',
 'C0627',
 'C0629',
 'C0238',
 'C0278',
 'C0515',
 'C0516',
 'C0115',
 'C0179',
 'C0240',
 'C0030',
 'C0924',
 'C0925',
 'C0926',
 'C0622',
 'C0249',
 'C0208',
 'C0028',
 'C0029',
 'C2096',
 'C0489',
 'C1158',
 'C1169',
 'C2600',
 'C2412',
 'C0023',
 'C0025',
 'C2168',
 'C0710',
 'C0443',


In [158]:
discouraged_concepts.set_index('legacyId').loc['C3324']['label']

'house of origin'

In [159]:
found = [('table','legacyId','column_with_concept','discouraged_concept_id','discouraged_concept_label')]

for t, tt in tables.items():
    #df = tables['resources']
    df = tables[t]
    #mask = np.column_stack([df[col].str.contains(r"C00", na=False) for col in df])
    mask = np.column_stack([df[col].isin(discouraged_concepts_ids_list) for col in df])
    result = df.loc[mask.any(axis=1)]
    if not result.empty:
        #found[t] = []
        print(t)
        lids = result['legacyId'].to_list()
        print(lids)
        for l in lids:
            target = df[df['legacyId'] == l]
            for c in target.columns:
                #print(target[c].iloc[0])
                if c=="legacyId":
                    continue
                if t=="concepts":
                    if target['status'].iloc[0] == "discouraged":
                        continue
                if target[c].iloc[0] in discouraged_concepts_ids_list:
                    found.append((t,l,c, target[c].iloc[0],discouraged_concepts.set_index('legacyId').loc[target[c].iloc[0]]['label']))



R0031_locations
['L0275_R0031', 'L0442_R0031']
actions
['A0182', 'A0030', 'A0104']
concepts
['C2090', 'C0408', 'C0409', 'C0410', 'C0948', 'C0949', 'C0950', 'C3069', 'C0602', 'C0165', 'C0394', 'C0541', 'C0653', 'C0744', 'C0395', 'C2003', 'C0416', 'C0417', 'C2776', 'C0125', 'C0231', 'C0611', 'C0612', 'C0683', 'C0764', 'C0784', 'C0415', 'C0433', 'C0636', 'C0806', 'C0160', 'C0647', 'C0701', 'C1230', 'C1231', 'C1271', 'C1272', 'C1273', 'C0335', 'C0194', 'C0376', 'C0449', 'C0454', 'C0455', 'C0456', 'C0457', 'C0476', 'C0530', 'C0531', 'C0474', 'C0475', 'C0672', 'C0217', 'C0432', 'C0438', 'C0439', 'C0440', 'C0441', 'C0673', 'C0860', 'C0863', 'C0864', 'C1811', 'C0822', 'C1975', 'C2774', 'C0494', 'C0495', 'C0205', 'C0206', 'C0207', 'C0498', 'C0505', 'C0518', 'C0519', 'C0520', 'C2087', 'C0203', 'C0496', 'C0497', 'C0502', 'C0503', 'C0421', 'C0422', 'C0174', 'C0176', 'C0183', 'C0184', 'C0175', 'C0177', 'C2091', 'C0210', 'C0499', 'C0500', 'C1112', 'C0628', 'C0630', 'C0180', 'C0219', 'C0525', 'C0529'

In [160]:
rdf = pd.DataFrame(found)
new_header = rdf.iloc[0] #grab the first row for the header
rdf = rdf[1:] #take the data less the header row
rdf.columns = new_header
rdf

,table,legacyId,column_with_concept,discouraged_concept_id,discouraged_concept_label
1,R0031_locations,L0275_R0031,class_id2,C0851,locus
2,R0031_locations,L0442_R0031,class_id2,C0851,locus
3,actions,A0182,related_entity_id,C0165,extra tempus gracie
4,actions,A0030,subject_semantic_id,C1465,adversary
5,actions,A0030,actant1_semantic_id,C1465,adversary
...,...,...,...,...,...
199,concepts,C2149,grammatical_number_id,C0954,singular
200,concepts,C2263,components_id,C2265,apostolici
201,concepts,C2264,plural_id,C2265,apostolici
202,concepts,C2264,grammatical_number_id,C0954,singular


In [161]:
d.write_df_to_gsheet("discouraged", "discouraged_concepts_2023", rdf)

20 2023-01-17 14:07:30 : Trying to write target dataset 'discouraged_concepts_2023' to gsheet handle: discouraged
20 2023-01-17 14:07:30 : Using existing output worksheet
20 2023-01-17 14:07:31 : Successfully written to gsheet handle: discouraged


In [ ]:
#mask = np.column_stack([df[col].str.contains(r"\^", na=False) for col in df])
#df.loc[mask.any(axis=1)]
